In [3]:
import pandas as pd

In [4]:
courses = pd.read_csv('../data/courses.csv')
assessments = pd.read_csv('../data/assessments.csv')
studentInfo = pd.read_csv('../data/studentInfo.csv')
studentAssessment = pd.read_csv('../data/studentAssessment.csv')

print('courses')
display(courses.head())
print('assessments')
display(assessments.head())
print('studentInfo')
display(studentInfo.head())
print('studentAssessment')
display(studentAssessment.head())

courses


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


assessments


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


studentInfo


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


studentAssessment


,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [44]:
courses.loc[courses.code_module == 'DDD']

,code_module,code_presentation,module_presentation_length
8,DDD,2013J,261
9,DDD,2014J,262
10,DDD,2013B,240
11,DDD,2014B,241


In [5]:
{'CCC': ['2014J', '2014B'],
 'DDD': ['2013J', '2014J', '2013B', '2014B']}

{'CCC': ['2014J', '2014B'], 'DDD': ['2013J', '2014J', '2013B', '2014B']}

In [6]:
all_exam_id_assessment = assessments.loc[assessments['assessment_type']=='Exam'].set_index(['code_module', 'code_presentation'])
display(all_exam_id_assessment.loc[['CCC', 'DDD']])

id_assessment assessment_type   date  weight
code_module code_presentation                                              
CCC         2014B                      24290            Exam    NaN   100.0
            2014B                      40087            Exam    NaN   100.0
            2014J                      24299            Exam    NaN   100.0
            2014J                      40088            Exam    NaN   100.0
DDD         2013B                      25340            Exam  240.0   100.0
            2013J                      25354            Exam  261.0   100.0
            2014B                      25361            Exam  241.0   100.0
            2014J                      25368            Exam    NaN   100.0

In [7]:
def get_module_presentation_data(code_module,
                    code_presentation,
                    exam_id_assessment,
                    first_try=True,
                    drop_unregistered=True,
                    have_final_result=True,
                    if_exam_keep_written=True):
    
    studentInfo_tmp = studentInfo.copy()
#     filtering module
    studentInfo_tmp = studentInfo_tmp.loc[studentInfo_tmp.code_module==code_module]
#     filtering module presentation
    studentInfo_tmp = studentInfo_tmp.loc[studentInfo_tmp.code_presentation==code_presentation]
#     dropping multipe attempts
    studentInfo_tmp.drop(labels=studentInfo_tmp.loc[studentInfo_tmp.num_of_prev_attempts!=0].index,inplace=True)
#     dropping Withrawn
    studentInfo_tmp.drop(labels=studentInfo_tmp.loc[studentInfo_tmp.final_result=='Withdrawn'].index,inplace=True)
    studentInfo_tmp.drop(columns=['num_of_prev_attempts'], inplace=True)
    studentInfo_tmp = studentInfo_tmp.set_index(['id_student'])
    
    studentAssessment_tmp = studentAssessment.copy()
    studentAssessment_tmp = studentAssessment_tmp.set_index(['id_assessment','id_student']).loc[exam_id_assessment]
    studentAssessment_tmp.drop(columns=['is_banked'], inplace=True)
    
    return studentInfo_tmp.merge(studentAssessment_tmp, how='left', left_index=True, right_index=True)

In [20]:
studentData_DDD_2013B = get_module_presentation_data('DDD','2013B',25340)
print(studentData_DDD_2013B.shape)
display(studentData_DDD_2013B.head())
studentData_DDD_2013J = get_module_presentation_data('DDD','2013J',25354)
print(studentData_DDD_2013J.shape)
display(studentData_DDD_2013J.head())
studentData_DDD_2014B = get_module_presentation_data('DDD','2014B',25361)
print(studentData_DDD_2014B.shape)
display(studentData_DDD_2014B.head())
studentData_DDD_2014J = get_module_presentation_data('DDD','2014J',25368)
print(studentData_DDD_2014J.shape)
display(studentData_DDD_2014J.head())

(703, 12)


,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,date_submitted,score
id_student,,,,,,,,,,,,
41060,DDD,2013B,M,Ireland,HE Qualification,70-80%,0-35,60,N,Fail,NaN,NaN
53488,DDD,2013B,F,East Anglian Region,HE Qualification,90-100%,0-35,60,N,Pass,231.0,73.0
55242,DDD,2013B,F,North Western Region,Lower Than A Level,30-40%,0-35,60,N,Fail,NaN,NaN
60135,DDD,2013B,F,West Midlands Region,A Level or Equivalent,NaN,0-35,60,N,Pass,229.0,40.0
66579,DDD,2013B,M,South Region,Lower Than A Level,40-50%,35-55,60,N,Fail,NaN,NaN


(1071, 12)


,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,date_submitted,score
id_student,,,,,,,,,,,,
27935,DDD,2013J,F,East Anglian Region,Lower Than A Level,30-40%,0-35,60,Y,Fail,NaN,NaN
28046,DDD,2013J,F,Yorkshire Region,HE Qualification,10-20,35-55,70,N,Fail,237.0,40.0
29411,DDD,2013J,M,East Midlands Region,A Level or Equivalent,80-90%,0-35,60,N,Pass,237.0,62.0
33930,DDD,2013J,F,South West Region,A Level or Equivalent,20-30%,0-35,90,N,Pass,237.0,67.0
34653,DDD,2013J,M,East Midlands Region,Lower Than A Level,30-40%,0-35,60,N,Fail,NaN,NaN


(584, 12)


,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,date_submitted,score
id_student,,,,,,,,,,,,
39660,DDD,2014B,F,North Western Region,Lower Than A Level,0-10%,0-35,120,N,Fail,NaN,NaN
46753,DDD,2014B,F,East Anglian Region,A Level or Equivalent,60-70%,35-55,60,N,Pass,234.0,36.0
50476,DDD,2014B,F,South West Region,HE Qualification,0-10%,0-35,60,N,Distinction,234.0,87.0
51030,DDD,2014B,M,West Midlands Region,A Level or Equivalent,60-70%,0-35,60,N,Pass,236.0,73.0
58617,DDD,2014B,M,East Midlands Region,A Level or Equivalent,70-80%,0-35,60,N,Fail,NaN,NaN


(958, 12)


,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,date_submitted,score
id_student,,,,,,,,,,,,
51033,DDD,2014J,F,South Region,A Level or Equivalent,80-90%,0-35,60,N,Pass,243.0,53.0
51153,DDD,2014J,F,East Midlands Region,HE Qualification,30-40%,35-55,60,N,Pass,243.0,53.0
51955,DDD,2014J,M,Scotland,A Level or Equivalent,80-90%,0-35,60,Y,Fail,242.0,22.0
68417,DDD,2014J,M,Scotland,Lower Than A Level,70-80%,0-35,60,N,Pass,243.0,58.0
71676,DDD,2014J,F,East Midlands Region,Lower Than A Level,80-90%,0-35,60,N,Pass,243.0,47.0


In [21]:
studentData_DDD_2013B.loc[studentData_DDD_2013B.score.isnull()].loc[studentData_DDD_2013B.final_result=='Pass'].index.values

array([], dtype=int64)

In [22]:
print(studentData_DDD_2013J.loc[studentData_DDD_2013J.score.isnull()].loc[studentData_DDD_2013J.final_result=='Pass'].index.values)
studentData_DDD_2013J.drop(labels=[592315], inplace=True)
print(studentData_DDD_2013J.loc[studentData_DDD_2013J.score.isnull()].loc[studentData_DDD_2013J.final_result=='Pass'].index.values)

[592315]
[]


In [23]:
studentData_DDD_2014B.loc[studentData_DDD_2014B.score.isnull()].loc[studentData_DDD_2014B.final_result=='Pass'].index.values

array([], dtype=int64)

In [24]:
studentData_DDD_2014J.loc[studentData_DDD_2014J.score.isnull()].loc[studentData_DDD_2014J.final_result=='Pass'].index.values

array([], dtype=int64)

In [31]:
display(assessments.set_index(['code_module', 'code_presentation']).sort_index().loc[('DDD', '2013B')])
display(assessments.set_index(['code_module', 'code_presentation']).sort_index().loc[('DDD', '2013J')])
display(assessments.set_index(['code_module', 'code_presentation']).sort_index().loc[('DDD', '2014B')])
display(assessments.set_index(['code_module', 'code_presentation']).sort_index().loc[('DDD', '2014J')])

id_assessment assessment_type   date  weight
code_module code_presentation                                              
DDD         2013B                      25341             CMA   23.0     2.0
            2013B                      25342             CMA   51.0     3.0
            2013B                      25343             CMA   79.0     3.0
            2013B                      25344             CMA  114.0     4.0
            2013B                      25345             CMA  149.0     4.0
            2013B                      25346             CMA  170.0     3.0
            2013B                      25347             CMA  206.0     6.0
            2013B                      25334             TMA   25.0     7.5
            2013B                      25335             TMA   53.0    10.0
            2013B                      25336             TMA   81.0    12.5
            2013B                      25337             TMA  116.0    15.0
            2013B                      25338             TMA  151.0    15.0
            2013B                      25339             TMA  200.0    15.0
            2013B                      25340            Exam  240.0   100.0

id_assessment assessment_type   date  weight
code_module code_presentation                                              
DDD         2013J                      25348             TMA   25.0    10.0
            2013J                      25349             TMA   53.0    12.5
            2013J                      25350             TMA   88.0    17.5
            2013J                      25351             TMA  123.0    20.0
            2013J                      25352             TMA  165.0    20.0
            2013J                      25353             TMA  207.0    20.0
            2013J                      25354            Exam  261.0   100.0

id_assessment assessment_type   date  weight
code_module code_presentation                                              
DDD         2014B                      25355             TMA   25.0    10.0
            2014B                      25356             TMA   53.0    12.5
            2014B                      25357             TMA   74.0    17.5
            2014B                      25358             TMA  116.0    20.0
            2014B                      25359             TMA  158.0    20.0
            2014B                      25360             TMA  200.0    20.0
            2014B                      25361            Exam  241.0   100.0

id_assessment assessment_type   date  weight
code_module code_presentation                                              
DDD         2014J                      25362             TMA   20.0     5.0
            2014J                      25363             TMA   41.0    10.0
            2014J                      25364             TMA   62.0    10.0
            2014J                      25365             TMA  111.0    25.0
            2014J                      25366             TMA  146.0    25.0
            2014J                      25367             TMA  195.0    25.0
            2014J                      25368            Exam    NaN   100.0

In [32]:
studentData_DDD = pd.concat([studentData_DDD_2013B, studentData_DDD_2013J, studentData_DDD_2014B, studentData_DDD_2014J], axis=0)

In [40]:
print(studentData_DDD.shape)
display(studentData_DDD.head())

(3315, 12)


,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,date_submitted,score
id_student,,,,,,,,,,,,
41060,DDD,2013B,M,Ireland,HE Qualification,70-80%,0-35,60,N,Fail,NaN,NaN
53488,DDD,2013B,F,East Anglian Region,HE Qualification,90-100%,0-35,60,N,Pass,231.0,73.0
55242,DDD,2013B,F,North Western Region,Lower Than A Level,30-40%,0-35,60,N,Fail,NaN,NaN
60135,DDD,2013B,F,West Midlands Region,A Level or Equivalent,NaN,0-35,60,N,Pass,229.0,40.0
66579,DDD,2013B,M,South Region,Lower Than A Level,40-50%,35-55,60,N,Fail,NaN,NaN


In [43]:
path = '../data/studentData_DDD_all_yrs.csv'
studentData_DDD.to_csv(path_or_buf=path)